# Local Search
## Eight Queens
### Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

### Eight Queens Code
Before we attempt any algorithmic solution, we need a way to model the Eight Queens puzzle: the board, the valid moves, and so on.

We could use a 2D array to represent the chessboard. However we get some automatic benefits if we use another representation. Specifically, we will use a 1D array of size 8. Each cell in the array stores the location of the queen in that column. This means the board state itself enforces one of the rules of the game, it is not possible to put two queens in one column.

So the following board:
```
   01234567
7  .X......
6  .....X..
5  X.......
4  ..X.....
3  .......X
2  ....X...
1  ......X.
0  ...X....
```
would be represented by the following array:
`[5, 7, 4, 0, 2, 6, 1, 3]`

We'll use numpy for this example, so it will actually be:

In [1]:
import numpy as np
np.array([5, 7, 4, 0, 2, 6, 1, 3], dtype=np.int)

array([5, 7, 4, 0, 2, 6, 1, 3])

Notice that the board above is not a valid arrangement. The two queens highlighted with `O` symbols are attacking each other.

```
   01234567
7  .X......
6  .....X..
5  X.......
4  ..O.....
3  .......X
2  ....O...
1  ......X.
0  ...X....
```

Being able to actually model the problem we are trying to solve is a necessary part of writing AI code. You can spend surprisingly long just implementing the rules of the puzzle, but without this step you cannot hope to solve it! Sometimes we will provide code to help, as in the Tower of Hanoi example. But you will often need to modify or create your own from scratch. So it's important to take some time to understand how this code works.

The code below implements the board state for the eight queens puzzle. It includes a method which can calculate the cost heuristic – in this case, the number of pairs of attacking queens. It also has methods which generate neighbouring states, which are states with exactly one queen moved within its column.

In [2]:
class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, state=None, n=8):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            state = np.random.randint(0, n, n)
        else:
            self.n = len(state)
        self.state = state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))

    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += np.any(self.state[i + 1:] == self.state[i])

            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += np.any(self.state[i + 1:] == upper_diagonal)
            count += np.any(self.state[i + 1:] == lower_diagonal)
        return count

    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0

    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state} cost {self.cost()}"

Read the code before moving on. You should at least understand what each method does (e.g. from its name, parameters, and docstring) but you may also want to read some of the implementation.

In [9]:
state = EightQueensState()
print(state)
print(state.is_goal())

[2 5 5 3 4 2 5 2] cost 7
False


### Hill Climbing
Hill-climbing search (Section 4.1.1 in R&N) is a very general and intuitive class of algorithms. Hill-climbing methods can be applied if one is able to assign a _value_ to each state on the search path. The algorithm then continuously moves towards higher-valued states (that is, climbing uphill). The algorithm terminates if there is no higher-valued state in the current neighbourhood.

<figure>
<img src="resources/hill_climbing_no_capt.png", width=600>
<center>Figure 1. Pseudocode of the hill-climbing algorithm taken from Russell and Norvig (p.122). <br> At each step the current node is replaced by the best neighbour.</center>
</figure>
<br>

Notice that since we are using a *cost function* for our heuristic (smaller values are good), what we really want to do is climb *“downhill”*. We will adapt the pseudocode to achieve this.

We already have our state representation, so now let's write the code which does the hill climbing.

In [13]:
class HillClimber:
    """Applies the hill climbing algorithm to solve the 8 queens puzzle"""
    def __init__(self, state=EightQueensState()):
        self.state = state
        self.n = state.n

    @staticmethod
    def best_neighbour(state):
        """Gets all neighbours from the state, then returns the one with lowest cost"""
        return min(state.neighbourhood(), key=lambda x: x.cost())

    def hill_climb(self):
        """
        Repeatedly take the best neighbour of each state until we find a solution (or get stuck)
        :returns a goal state OR a local minimum (if all options are worse than the current one but it is not a goal)
        """
        if self.state.is_goal():
            return self.state

        while True:
            neighbour = HillClimber.best_neighbour(self.state)
            if neighbour.is_goal():
                return neighbour
            if neighbour.cost() < self.state.cost():
                self.state = neighbour
            else:
                # notice the method gives up if there are no better options
                return self.state

state = EightQueensState()
climber = HillClimber(state)
solution = climber.hill_climb()
print(solution)

Goal state! [7 3 0 2 5 1 6 4]


As always, take some time to read and fully understand the code.

When I ran the code above, the following solution was found: `[3 1 7 5 0 2 4 6]`. Here it is visualised on a board:
```
   01234567
7  ..X.....
6  .......X
5  ...X....
4  ......X.
3  X.......
2  .....X..
1  .X......
0  ....X...
```

Looks great, right! But there's a problem. Try running the solver again, and you might get something like this:

In [11]:
state = EightQueensState()
climber = HillClimber(state)
solution = climber.hill_climb()
print(solution)

[3 6 6 2 0 6 4 7] cost 2


This time the output is: `[3 6 6 2 0 6 4 7] cost 2`

We can see immediately that this is not a solution, there are two queens on row 6. The hill climbing algorithm gave up! The problem was that no neighbouring state was able to improve the result. Every possible way of moving a single queen on this board results in a state with cost 2 or higher.

This is called a *local minimum*, and it's a common flaw with hill climbing algorithms. We know this isn't a *global minimum*, because we know there are states with cost 0, but we cannot see it from where we are in the search space.

We can make the code more likely to succeed by changing this line
```python
if neighbour.cost() < self.state.cost():
```
to
```python
if neighbour.cost() <= self.state.cost():
```

This actually follows the pseudocode above more closely. But this is still not guaranteed to avoid the problem, some local minimum states might be strictly better than all other states in their neighbourhood. And worse, it could result in an infinite loop where the algorithm repeatedly swaps between two equal cost states, both at the bottom of a valley (or the top of a hill), which we will never escape.

### Task: Iterative Hill Climb
Luckily, there is a simple fix for this problem, which is to perform hill climbing iteratively. Start at a random location, hill climb, then if we hit a local minimum or maximum, we restart from another random location. It might seem like a hacky solution but it's surprisingly effective.

* Modify the code below to include an iterative version of the hill climbing algorithm
* Once you have done that, add some metrics to measure how many steps this algorithm takes (you pick, e.g. how many states are generated, how many 'next best states' are followed, how many times does it restart)
* $n$-queens has solutions for all values of $n$ greater than $3$, how well does your code fare on a 10x10 board? How about 15x15 or 20x20? (Note you can call `EightQueensState(n=15)`)
* For an additional challenge, read about and then implement another local search method, either from the textbook or elsewhere online. I suggest Simulated Annealing or Genetic Algorithms. Compare your results to iterative hill climbing on the same metric, and post your best result to the forum!

In [ ]:
class HillClimber:
    """Applies the hill climbing algorithm to solve the 8 queens puzzle"""
    def __init__(self, state=EightQueensState()):
        self.state = state
        self.n = state.n

    @staticmethod
    def best_neighbour(state):
        """Gets all neighbours from the state, then returns the one with lowest cost"""
        return min(state.neighbourhood(), key=lambda x: x.cost())

    def hill_climb(self):
        """
        Repeatedly take the best neighbour of each state until we find a solution (or get stuck)
        :returns a goal state OR a local minimum (if all options are worse than the current one but it is not a goal)
        """
        if self.state.is_goal():
            return self.state

        while True:
            neighbour = HillClimber.best_neighbour(self.state)
            if neighbour.is_goal():
                return neighbour
            if neighbour.cost() < self.state.cost():
                self.state = neighbour
            else:
                # notice the method gives up if there are no better options
                return self.state

    def iterative_hill_climb(self):
        """Your code goes here!"""
        pass
    
state = EightQueensState()
climber = HillClimber(state)
solution = climber.iterative_hill_climb()
print(solution)